<a href="https://colab.research.google.com/github/rar8393/NLG_tests/blob/main/roberta_model_legal_finetune_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://github.com/huggingface/notebooks/blob/main/examples/language_modeling.ipynb

In [1]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 32.3 MB/s 
     |████████████████████████████████| 5.3 MB 51.4 MB/s 
     |████████████████████████████████| 115 kB 72.8 MB/s 
     |████████████████████████████████| 212 kB 72.9 MB/s 
     |████████████████████████████████| 163 kB 71.0 MB/s 
     |████████████████████████████████| 127 kB 61.7 MB/s 
     |████████████████████████████████| 7.6 MB 59.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
import transformers

print(transformers.__version__)

4.23.1


In [3]:
from datasets import load_dataset
datasets = load_dataset("text", data_files={"train": 'train.txt', "validation": 'val.txt'})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-becfac4354a464f7/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [5]:
show_random_elements(datasets["train"])

,text
0,"In maintaining its opposition to the application, counsel for the Commonwealth has submitted that Order 52, r 14 requires in terms that ""each party to the proceeding in the court appealed from ... shall be joined as a party ... to the appeal"" and he relies upon the observations of the Full Court in United States Tobacco v Minister for Consumer Affairs (1988) 20 FCR 520 where it was said, at 534: ""An intervener, whether pursuant to s 12 of the ADJR Act, O 6, r 8(1) of the Federal Court Rules , s 78A of the Judiciary Act 1903 (Cth) or otherwise , becomes a party to the proceedings with the benefits and burdens of that status."" (Emphasis added). 5 The proposition that an intervener becomes a party to the proceedings, the Second Respondent pointed out, also was accepted in Cheesman v Waters (1997) 77 FCR 221 at 227, another decision of the Full Court. Moreover, in United States Tobacco the Court referred with approval to the following passage from the judgment of Hutley JA, with whom Reynolds and Glass JJA agreed, in Corporate Affairs Commission v Bradley [1974] 1 NSWLR 391, 396: ""A person accepted as an intervener becomes a party to the proceedings with all the privileges of a party. Thus he can appeal, tender evidence and participate fully in all aspects of the argument. His position is quite different from that of an amicus curiae. Interveners have been allowed to appeal. Thus the Attorney-General of the Commonwealth appealed to the Privy Council in the Boilermakers' case ..., though he was only an intervener in the High Court..."""
1,"6.4 ACCEPTANCE. Each Deliverable shall be deemed accepted upon the earlier of: (i) receipt of written notice of acceptance from HSNS, or (ii) ten (10) days after delivery of such Deliverable (""Acceptance Period""), unless HSNS provides E.piphany with a detailed list of material non-conformities during the Acceptance Period. If HSNS properly rejects a Deliverable, E.piphany will use reasonable commercial efforts to correct the material non-conformity specified in the notice. When it believes that it has made the necessary corrections, E.piphany will again deliver the Deliverable to HSNS and the acceptance/ rejection/correction process shall be reapplied until the Deliverable substantially complies with the requirements of the Statement of Work."
2,"a) Client will be promoted as the preferred and dominant reseller of Vitamins in the Shopping Channel on the WebCrawler Site during the term of this Agreement. As such, Excite may not display banner advertising and/or promotional placements for any of Client's Competitors that are, in the aggregate, of equal or greater prominence and exposure than the aggregate of Client's links, advertising banners and promotional placements on the pages of the Shopping Channel or the WebCrawler Site. Notwithstanding the foregoing, Excite may display links to Excite's own products and services anywhere in the Excite Network, and may display links to Client's Competitors in results pages of search services in response to user queries, in general directories of Web sites, in classified advertising listings and in results in the ""Jango"" shopping search service throughout the Excite Network."
3,"11.7 Assignment. Company may not assign or delegate this Agreement or any of its rights or obligations hereunder, whether voluntarily, involuntarily, by operation of Law or otherwise, without ACSI's prior written consent, which consent shall not be unreasonably withheld or delayed, except that Company may assign this Agreement to any direct or indirect wholly owned subsidiary in connection with any corporate reorganization undertaken for the purpose of minimizing the tax liability of Company and its Affiliates or other bona fide corporate purpose or in connection with any Change of Control [***]. Subject to the preceding two sentences, this Agreement shall be binding upon and inure to the benefit of the Parties hereto and their successors and assigns. ACSI may assi

In [6]:
#"facebook/opt-125m". "saibo/legal-roberta-base"
model_checkpoint = "roberta-base"

In [7]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [8]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [9]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#0:   0%|          | 0/7 [00:00<?, ?ba/s]

#1:   0%|          | 0/7 [00:00<?, ?ba/s]

#2:   0%|          | 0/7 [00:00<?, ?ba/s]

#3:   0%|          | 0/7 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (530 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (724 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (714 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (546 > 512). Running this sequence through the model will result in indexing errors


#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (561 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (678 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (561 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (574 > 512). Running this sequence through the model will result in indexing errors


In [10]:
# block_size = tokenizer.model_max_length
block_size = 512

In [11]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [12]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/7 [00:00<?, ?ba/s]

#1:   0%|          | 0/7 [00:00<?, ?ba/s]

#2:   0%|          | 0/7 [00:00<?, ?ba/s]

#3:   0%|          | 0/7 [00:00<?, ?ba/s]

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

In [13]:
from transformers import AutoModelForCausalLM
#If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, is_decoder=True)

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

In [14]:
from transformers import Trainer, TrainingArguments

In [18]:
#https://stackoverflow.com/questions/72680734/huggingface-trainer-only-doing-3-epochs-no-matter-the-trainingarguments
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-legal-test-1",
    evaluation_strategy = "steps", #epoch
    num_train_epochs=6, 
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
    warmup_steps=1000,                # number of warmup steps for learning rate  
    save_steps=2000,
    eval_steps=500,
    save_total_limit=10,
    load_best_model_at_end= True,
    #output_dir='./results',
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [20]:
trainer.train()

***** Running training *****
  Num examples = 13704
  Num Epochs = 6
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 10278


Step,Training Loss,Validation Loss
500,3.275500,3.088559
1000,3.171500,3.032196
1500,3.057500,2.956081
2000,3.054100,2.897416
2500,2.990100,2.850943
3000,2.938200,2.815245
3500,2.894300,2.777779
4000,2.829000,2.757435
4500,2.812800,2.732371
5000,2.799100,2.711157


***** Running Evaluation *****
  Num examples = 2402
  Batch size = 8
***** Running Evaluation *****
  Num examples = 2402
  Batch size = 8
***** Running Evaluation *****
  Num examples = 2402
  Batch size = 8
***** Running Evaluation *****
  Num examples = 2402
  Batch size = 8
Saving model checkpoint to roberta-base-legal-test-1/checkpoint-2000
Configuration saved in roberta-base-legal-test-1/checkpoint-2000/config.json
Model weights saved in roberta-base-legal-test-1/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2402
  Batch size = 8
***** Running Evaluation *****
  Num examples = 2402
  Batch size = 8
***** Running Evaluation *****
  Num examples = 2402
  Batch size = 8
***** Running Evaluation *****
  Num examples = 2402
  Batch size = 8
Saving model checkpoint to roberta-base-legal-test-1/checkpoint-4000
Configuration saved in roberta-base-legal-test-1/checkpoint-4000/config.json
Model weights saved in roberta-base-legal-test-1/checkpoint-4000/

TrainOutput(global_step=10278, training_loss=2.827889844215573, metrics={'train_runtime': 14230.6118, 'train_samples_per_second': 5.778, 'train_steps_per_second': 0.722, 'total_flos': 2.1646739454050304e+16, 'train_loss': 2.827889844215573, 'epoch': 6.0})

In [21]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("roberta-base-legal-test-1/checkpoint-10000")

loading configuration file roberta-base-legal-test-1/checkpoint-10000/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base-legal-test-1/checkpoint-10000",
  "architectures": [
    "RobertaForCausalLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": true,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.23.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file roberta-base-legal-test-1/checkpoint-10000/pytorch_model.bin
All model checkpoint weights were used when initializing RobertaForCausalLM.

All th

In [23]:
import importlib
import generate_txt
importlib.reload(generate_txt)
generator = generate_txt.generate_model_results(tokenizer, model)

In [24]:
test_metrics_1 = generator.generate_sample("The event has a ", visualize=False)

/content/generated_text_metrics.py:52: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity.append(gen_doc.similarity(sent))
/content/generated_text_metrics.py:54: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Span.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one 

In [25]:
for key in test_metrics_1.keys():
  print(key)

The event has a 
This Agreement may be terminated by Franchisee upon written request, by Vapotherm on [**] following written notice to Pretzel Time to the extent Pretzel Time has entered into an agreement with Franchisee not to renew this Agreement that shall not require Franchisee to have any obligation to the Franchisee thereunder
9.2 No Termination. For each year of this Agreement, the first annual Termination Period shall fail after the end of the first  Quarter of the Fiscal Year and a replacement agreement for the first year for the following Calendar Years shall be entered into by the Parties; provided, however, that if the expiration
The fact that it was necessary for the Commissioner to make any orders pursuant to s 424A and also the fact that it had already received a notice from all the relevant Governmental Authorities, I am of opinion that this submission must be rejected: ACCC v Minister for Immigration and Multicultural Affairs [2004] FCA
This Agreement shall become effe

In [26]:
for key in test_metrics_1.keys():
  print(test_metrics_1[key]['metric'])

{'number_ents': 0, 'ents': [], 'number_nouns': 1, 'nouns': ['event'], 'number_sents': 1, 'avg_sent_len': 4.0, 'sent_repeat_score': [0.0]}
{'number_ents': 7, 'ents': [Franchisee, Vapotherm, Pretzel Time, Pretzel Time, Franchisee, Franchisee, Franchisee], 'number_nouns': 17, 'nouns': ['Agreement', 'Franchisee', 'request', 'Vapotherm', 'notice', 'Pretzel', 'Time', 'extent', 'Pretzel', 'Time', 'agreement', 'Franchisee', 'Agreement', 'Franchisee', 'obligation', 'Franchisee', 'thereunder'], 'number_sents': 1, 'avg_sent_len': 54.0, 'sent_repeat_score': [0.31481481481481477]}
{'number_ents': 4, 'ents': [each year, first, the first year, Calendar Years], 'number_nouns': 17, 'nouns': ['No', 'termination', 'year', 'Agreement', 'Termination', 'Period', 'end', 'Quarter', 'Fiscal', 'Year', 'replacement', 'agreement', 'year', 'Calendar', 'year', 'party', 'expiration'], 'number_sents': 2, 'avg_sent_len': 29.5, 'sent_repeat_score': [0.0, 0.32727272727272727]}
{'number_ents': 2, 'ents': [Governmental Au

In [27]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in test_metrics_1.keys():
  print(test_metrics_1[key]['metric']['number_ents'], test_metrics_1[key]['metric']['number_nouns'],
        test_metrics_1[key]['metric']['number_sents'],  test_metrics_1[key]['metric']['avg_sent_len'],
        test_metrics_1[key]['metric']['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
0 1 1 4.0 [0.0]
7 17 1 54.0 [0.31481481481481477]
4 17 2 29.5 [0.0, 0.32727272727272727]
2 17 1 58.0 [0.2068965517241379]
4 13 2 27.0 [0.0, 0.2272727272727273]
4 13 3 20.333333333333332 [0.0, 0.31034482758620685, 0.0]


In [28]:
test_metrics_2 = generator.generate_sample("The conference started with a ", visualize=False)

In [29]:
for key in test_metrics_2.keys():
  print(key)

The conference started with a 
2.5.2 During the Term, IBM shall, in the event of any material change in a                    party  is able, without a request of third             
Counsel for the applicant also relied upon the decision of Griffith J in Re Minister for Immigration | Appellant Appellant; Minister for Immigration and Multicultural Affairs; Ex parte Miah [2006] HCA 2 at [44] and [58] of the Federal Court Rules 1976 (Cth):
8. Representations and Warranties of the Transaction Entities. If the transactions occur pursuant to this Agreement that result in any material change in the price of the Company's common stock, the Transaction Entities shall give such notice as follows: (i) no later than [***] from the date or time of
The Court has power to make rules on the construction of a statutory provision in accordance with an Act or statute. The High Court has established a power to grant a statutory provision under s 41B of the Act "and in respect of the same provision in Parti

In [30]:
for key in test_metrics_2.keys():
  print(test_metrics_2[key]['metric'])

{'number_ents': 0, 'ents': [], 'number_nouns': 1, 'nouns': ['conference'], 'number_sents': 1, 'avg_sent_len': 5.0, 'sent_repeat_score': [0.0]}
{'number_ents': 2, 'ents': [IBM, third], 'number_nouns': 7, 'nouns': ['Term', 'IBM', 'event', 'material', 'change', 'party', 'request'], 'number_sents': 2, 'avg_sent_len': 14.5, 'sent_repeat_score': [0.0, 0.2142857142857143]}
{'number_ents': 6, 'ents': [Multicultural Affairs, 44, 58, the Federal Court Rules, 1976, Cth], 'number_nouns': 23, 'nouns': ['counsel', 'applicant', 'decision', 'Griffith', 'J', 'Re', 'Minister', 'Immigration', '|', 'Appellant', 'Appellant', 'Minister', 'Immigration', 'Multicultural', 'Affairs', 'Ex', 'parte', 'Miah', 'HCA', 'Federal', 'Court', 'Rules', 'Cth'], 'number_sents': 1, 'avg_sent_len': 53.0, 'sent_repeat_score': [0.26415094339622647]}
{'number_ents': 4, 'ents': [8, Representations and Warranties of the Transaction Entities, Company, Transaction Entities], 'number_nouns': 17, 'nouns': ['Representations', 'Warranti

In [31]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in test_metrics_2.keys():
  print(test_metrics_2[key]['metric']['number_ents'], test_metrics_2[key]['metric']['number_nouns'],
        test_metrics_2[key]['metric']['number_sents'],  test_metrics_2[key]['metric']['avg_sent_len'],
        test_metrics_2[key]['metric']['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
0 1 1 5.0 [0.0]
2 7 2 14.5 [0.0, 0.2142857142857143]
6 23 1 53.0 [0.26415094339622647]
4 17 3 20.333333333333332 [0.0, 0.0, 0.1568627450980392]
3 20 2 28.5 [0.045454545454545414, 0.22857142857142854]
4 13 1 58.0 [0.39655172413793105]


In [32]:
#event_txt = 'Calling All Climate Warriors! Youth for Climate Justice is hosting a series of webinars where we feature three climate justice-focused initiatives spearheaded by young community organizers. Discover local movements in your community, learn about effective ways to communicate with legislators and lead the charge in climate justice! '
sick_txt = 'The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will '


In [33]:
test_metrics_3 = generator.generate_sample(sick_txt, visualize=False)

In [34]:
for key in test_metrics_3.keys():
  print(key)

The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
Confidential Treatment Requested by Company The Company represents and warrants to PNC that it has entered into and will perform its obligations under this Agreement with PNC in accordance with those terms and conditions and, in such performance, it maintains, and will, comply with and comply with, the rules and regulations that (a
(b) By Accuray, Inc.    10.1. The Assay will be effective as of the Effective Date and shall remain the   original  and  exclusive  remedy  and,  except as  expressly allowed  by this Agreement,  subject to the provisions  of this
b)     Company  shall ensure that its  operations and the  management and  other  aspects  of  them  maintain current  quality and  compliance  with the provisions  of this  Agreement,  including  quality control, but excluding any applicable Laws,  but  excluding all applicable
5.1       No  Representations or Warrant

In [35]:
for key in test_metrics_3.keys():
  print(test_metrics_3[key]['metric'])

{'number_ents': 1, 'ents': [this year], 'number_nouns': 4, 'nouns': ['flu', 'year', 'conference', 'vaccine'], 'number_sents': 3, 'avg_sent_len': 7.0, 'sent_repeat_score': [0.0, 0.0, 0.0]}
{'number_ents': 0, 'ents': [], 'number_nouns': 15, 'nouns': ['Confidential', 'Treatment', 'Company', 'Company', 'warrant', 'PNC', 'obligation', 'Agreement', 'PNC', 'accordance', 'term', 'condition', 'performance', 'rule', 'regulation'], 'number_sents': 1, 'avg_sent_len': 59.0, 'sent_repeat_score': [0.35593220338983056]}
{'number_ents': 3, 'ents': [Accuray, Inc., 10.1, Assay], 'number_nouns': 8, 'nouns': ['Accuray', 'Inc.', 'Assay', 'Effective', 'Date', 'remedy', 'Agreement', 'provision'], 'number_sents': 4, 'avg_sent_len': 13.5, 'sent_repeat_score': [0.0, 0.0, 0.3076923076923077, 0.2222222222222222]}
{'number_ents': 1, 'ents': [Laws], 'number_nouns': 11, 'nouns': ['Company', 'operation', 'management', 'aspect', 'quality', 'compliance', 'provision', 'Agreement', 'quality', 'control', 'law'], 'number_se

In [36]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in test_metrics_3.keys():
  print(test_metrics_3[key]['metric']['number_ents'], test_metrics_3[key]['metric']['number_nouns'],
        test_metrics_3[key]['metric']['number_sents'],  test_metrics_3[key]['metric']['avg_sent_len'],
        test_metrics_3[key]['metric']['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
1 4 3 7.0 [0.0, 0.0, 0.0]
0 15 1 59.0 [0.35593220338983056]
3 8 4 13.5 [0.0, 0.0, 0.3076923076923077, 0.2222222222222222]
1 11 1 60.0 [0.43333333333333335]
3 7 1 28.0 [0.25]
3 13 2 29.5 [0.0, 0.2586206896551724]


In [37]:
event_txt = 'Calling All Climate Warriors! Youth for Climate Justice is hosting a series of webinars where we feature three climate justice-focused initiatives spearheaded by young community organizers. Discover local movements in your community, learn about effective ways to communicate with legislators and lead the charge in climate justice! '
sick_txt = 'The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will '
#generator2(event_txt + sick_txt)
combo = event_txt + sick_txt

In [38]:
test_metrics_4 = generator.generate_sample(combo, visualize=False)

In [39]:
for key in test_metrics_4.keys():
  print(key)

Calling All Climate Warriors! Youth for Climate Justice is hosting a series of webinars where we feature three climate justice-focused initiatives spearheaded by young community organizers. Discover local movements in your community, learn about effective ways to communicate with legislators and lead the charge in climate justice! The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
"Agreement" means the agreement signed by King and MMT that includes a written instrument, document, instrument, instrument, or arrangement of the Parties in writing, signed between King and MMT; provided that King shall prepare for the signing of such instrument any documents necessary for King to perform its obligations under this Agreement
12.2 Severability. Severability. Either party may, at its own expense, and in its sole discretion, enforce any and all of the provisions of this Agreement, whether oral or written, in a manner that is consis

In [40]:
for key in test_metrics_4.keys():
  print(test_metrics_4[key]['metric'])

{'number_ents': 2, 'ents': [three, this year], 'number_nouns': 23, 'nouns': ['Climate', 'Warriors', 'youth', 'Climate', 'Justice', 'series', 'webinar', 'climate', 'justice', 'initiative', 'community', 'organizer', 'movement', 'community', 'way', 'legislator', 'charge', 'climate', 'justice', 'flu', 'year', 'conference', 'vaccine'], 'number_sents': 6, 'avg_sent_len': 12.333333333333334, 'sent_repeat_score': [0.0, 0.0, 0.04347826086956519, 0.0, 0.0, 0.0]}
{'number_ents': 3, 'ents': [MMT, King, MMT], 'number_nouns': 20, 'nouns': ['agreement', 'agreement', 'King', 'MMT', 'instrument', 'document', 'instrument', 'instrument', 'arrangement', 'party', 'writing', 'King', 'MMT', 'King', 'signing', 'instrument', 'document', 'King', 'obligation', 'Agreement'], 'number_sents': 1, 'avg_sent_len': 60.0, 'sent_repeat_score': [0.35]}
{'number_ents': 2, 'ents': [12.2 Severability, the Federal Court of Ch], 'number_nouns': 15, 'nouns': ['Severability', 'severability', 'party', 'expense', 'discretion', 'pr

In [41]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in test_metrics_4.keys():
  metric_load = test_metrics_4[key]['metric']
  print(metric_load['number_ents'], metric_load['number_nouns'],
        metric_load['number_sents'],  metric_load['avg_sent_len'],
        metric_load['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
2 23 6 12.333333333333334 [0.0, 0.0, 0.04347826086956519, 0.0, 0.0, 0.0]
3 20 1 60.0 [0.35]
2 15 3 19.666666666666668 [0.0, 0.0, 0.2962962962962963]
5 16 1 63.0 [0.4444444444444444]
4 14 3 20.666666666666668 [0.23333333333333328, 0.19999999999999996, 0.0]
0 6 1 19.0 [0.21052631578947367]


In [42]:
#check different lengths
length_metric_1 = generator.generate_sample(sick_txt, max_length=32, visualize=False)
length_metric_2 = generator.generate_sample(sick_txt, max_length=64, visualize=False)
length_metric_3 = generator.generate_sample(sick_txt, max_length=128, visualize=False)

In [43]:
for key in length_metric_1.keys():
  print(key)
  print(len(key))

The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
114
3.2.4 Insurance. Manufacturer requires comprehensive health insurance to be required in connection with the manufacture and commercialization of the Products and any Product, including
184
"The JSC" means the "decision made by the Parties, each, by a majority or a majority of its members, upon the unanimous consent
127
(e) License is required for the provision of the Products in order to provide commercial distribution to Customers. In connection with the marketing, distribution and offering of
178
(b) At its own expense, the Licensor shall, at its own cost and expense, provide to the Company,  or the  Licensor
114
7.2 This Manufacturing Agreement is entered into as of August 14, 2009. This Manufacturing Agreement, which is incorporated herein into the Manufacturing Agreement, is effective
177


In [44]:
for key in length_metric_1.keys():
  print(length_metric_1[key]['metric'])

{'number_ents': 1, 'ents': [this year], 'number_nouns': 4, 'nouns': ['flu', 'year', 'conference', 'vaccine'], 'number_sents': 3, 'avg_sent_len': 7.0, 'sent_repeat_score': [0.0, 0.0, 0.0]}
{'number_ents': 0, 'ents': [], 'number_nouns': 9, 'nouns': ['Insurance', 'Manufacturer', 'health', 'insurance', 'connection', 'manufacture', 'commercialization', 'Products', 'product'], 'number_sents': 2, 'avg_sent_len': 13.0, 'sent_repeat_score': [0.0, 0.13043478260869568]}
{'number_ents': 0, 'ents': [], 'number_nouns': 7, 'nouns': ['JSC', 'decision', 'party', 'majority', 'majority', 'member', 'consent'], 'number_sents': 1, 'avg_sent_len': 29.0, 'sent_repeat_score': [0.3448275862068966]}
{'number_ents': 1, 'ents': [Customers], 'number_nouns': 11, 'nouns': ['e', 'license', 'provision', 'Products', 'order', 'distribution', 'customer', 'connection', 'marketing', 'distribution', 'offering'], 'number_sents': 2, 'avg_sent_len': 15.5, 'sent_repeat_score': [0.09523809523809523, 0.0]}
{'number_ents': 1, 'ents

In [45]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in length_metric_1.keys():
  metric_load = length_metric_1[key]['metric']
  print(metric_load['number_ents'], metric_load['number_nouns'],
        metric_load['number_sents'],  metric_load['avg_sent_len'],
        metric_load['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
1 4 3 7.0 [0.0, 0.0, 0.0]
0 9 2 13.0 [0.0, 0.13043478260869568]
0 7 1 29.0 [0.3448275862068966]
1 11 2 15.5 [0.09523809523809523, 0.0]
1 6 1 29.0 [0.3793103448275862]
3 7 2 14.5 [0.0, 0.2666666666666667]


In [46]:
for key in length_metric_2.keys():
  print(key)

The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
It is necessary to note that in certain circumstances a person is required to be shown to have been the subject of the offence, for the Tribunal in its judgment is informed of that circumstance ( Gummow JA in Esso Australian Pty Ltd v The National Medicines Labourers Union (2005) 223 ALR
Section 8.5.3. Expiration of Sub-Licenses. After the Effective Date, each Party will not be required for any renewal, revamp, or modification of this Agreement without the prior written consent of the other. If either Party fails to renew this Agreement, then any additional and/or additional
5.        Excite will make available to customer                                  participants and prospects prospects, including all information and information    
The respondents noted that there has been no public policy or legal support for the granting of interlocutory injunctions under Section 4

In [47]:
for key in length_metric_2.keys():
    print(key)
    print(len(key))

The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
114
It is necessary to note that in certain circumstances a person is required to be shown to have been the subject of the offence, for the Tribunal in its judgment is informed of that circumstance ( Gummow JA in Esso Australian Pty Ltd v The National Medicines Labourers Union (2005) 223 ALR
288
Section 8.5.3. Expiration of Sub-Licenses. After the Effective Date, each Party will not be required for any renewal, revamp, or modification of this Agreement without the prior written consent of the other. If either Party fails to renew this Agreement, then any additional and/or additional
292
5.        Excite will make available to customer                                  participants and prospects prospects, including all information and information    
165
The respondents noted that there has been no public policy or legal support for the granting of interlocutory injunctions

In [48]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in length_metric_2.keys():
  metric_load = length_metric_2[key]['metric']
  print(metric_load['number_ents'], metric_load['number_nouns'],
        metric_load['number_sents'],  metric_load['avg_sent_len'],
        metric_load['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
1 4 3 7.0 [0.0, 0.0, 0.0]
6 18 2 27.5 [0.28301886792452835, 0.0]
3 13 4 13.5 [0.0, 0.0, 0.16129032258064513, 0.0714285714285714]
1 7 2 10.5 [0.0, 0.16666666666666663]
6 18 1 55.0 [0.1636363636363637]
3 14 1 49.0 [0.26530612244897955]


In [49]:
for key in length_metric_3.keys():
  print(key)
  print(len(key))

The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
114
6.9 Severability. In the event that either party (i) defaults on any of its obligations pursuant to this Agreement or fails to cause for default to occur a default or a material breach thereof, or (ii) this  Agreement is terminated pursuant to other terms applicable to the receiving party or (iii) the other party gives the other party written notice of default at least one year after the expiration of such notice period. It being the intent of the Parties, neither Party shall be in breach of its obligations hereunder, nor may the other Party be held to violate any of the representations or warranties of the other Party in this
634
(iii)     In the event that any provision  of this  Agreement has  become               invalid or  unenforceable,  the  provisions  thereof  shall not  be deemed  invalid  nor unenforceable in relation to, or  enforceable  elsewhere elsewher

In [50]:
for key in length_metric_3.keys():
  print(length_metric_3[key]['metric'])

{'number_ents': 1, 'ents': [this year], 'number_nouns': 4, 'nouns': ['flu', 'year', 'conference', 'vaccine'], 'number_sents': 3, 'avg_sent_len': 7.0, 'sent_repeat_score': [0.0, 0.0, 0.0]}
{'number_ents': 4, 'ents': [at least one year, Party, Party, Party], 'number_nouns': 33, 'nouns': ['severability', 'event', 'party', 'i', 'default', 'obligation', 'Agreement', 'default', 'default', 'breach', 'ii', 'Agreement', 'term', 'party', 'iii', 'party', 'party', 'notice', 'default', 'year', 'expiration', 'notice', 'period', 'intent', 'Parties', 'Party', 'breach', 'obligation', 'hereunder', 'Party', 'representation', 'warranty', 'Party'], 'number_sents': 3, 'avg_sent_len': 41.0, 'sent_repeat_score': [0.0, 0.375, 0.35]}
{'number_ents': 0, 'ents': [], 'number_nouns': 9, 'nouns': ['iii', 'event', 'provision', 'Agreement', 'provision', 'relation', 'Agreement', 'provision', 'provision'], 'number_sents': 3, 'avg_sent_len': 26.666666666666668, 'sent_repeat_score': [0.0, 0.4423076923076923, 0.25]}
{'numb

In [51]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in length_metric_3.keys():
  metric_load = length_metric_3[key]['metric']
  print(metric_load['number_ents'], metric_load['number_nouns'],
        metric_load['number_sents'],  metric_load['avg_sent_len'],
        metric_load['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
1 4 3 7.0 [0.0, 0.0, 0.0]
4 33 3 41.0 [0.0, 0.375, 0.35]
0 9 3 26.666666666666668 [0.0, 0.4423076923076923, 0.25]
2 16 2 29.0 [0.19999999999999996, 0.37735849056603776]
9 25 3 40.333333333333336 [0.07692307692307687, 0.16666666666666663, 0.2777777777777778]
13 31 3 39.0 [0.21311475409836067, 0.2857142857142857, 0.1428571428571429]


In [ ]:
#https://huggingface.co/docs/transformers/v4.16.2/en/custom_datasets

In [ ]:
!cp -r roberta-base-health-test-1/checkpoint-12000/ roberta-health-test-v1/

In [ ]:
!zip -r roberta_health_v1.zip roberta-health-test-v1/

  adding: roberta-health-test-v1/ (stored 0%)
  adding: roberta-health-test-v1/checkpoint-12000/ (stored 0%)
  adding: roberta-health-test-v1/checkpoint-12000/rng_state.pth (deflated 27%)
  adding: roberta-health-test-v1/checkpoint-12000/pytorch_model.bin (deflated 7%)
  adding: roberta-health-test-v1/checkpoint-12000/scheduler.pt (deflated 49%)
  adding: roberta-health-test-v1/checkpoint-12000/trainer_state.json (deflated 82%)
  adding: roberta-health-test-v1/checkpoint-12000/config.json (deflated 50%)
  adding: roberta-health-test-v1/checkpoint-12000/training_args.bin (deflated 48%)
  adding: roberta-health-test-v1/checkpoint-12000/optimizer.pt (deflated 8%)


In [ ]:
from google.colab import files
files.download("roberta_health_v1.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>